## DATA DISCOVERY AND DATA QUALITY

In [ ]:
import pandas as pd

cars_initial = pd.read_csv('Australian Vehicle Prices.csv')
cars = cars_initial.copy()
cars.head()

In [ ]:
cars.info()

The dataframe contains 16734 entries. 6 columns have missing values, that need to be addressed: Car/Suv, Location, BodyType, Doors, Seats, Price.
Also the columns Engine, FuelConsumption, CylindersInEngine, Doors, Seats, Price are of type object. We can cast them to numeric type. Some of the columns will need some transformations 

Drop rows with no price

In [ ]:
cars = cars.dropna(subset=['Price'])
cars.info()

## Explore each category

### Brand

In [ ]:
list(cars['Brand'].unique())

This category has a lot of possible values. To use it, we will create a new category, Luxury, stating if the car sold is considered a luxury car depending on the brand. 

In [ ]:
def categorize_brand(brand):
    #  Built with chat-gpt help. To be discussed with a domain expert
    luxury_brands = ['MG', 'Land', 'Audi', 'BMW', 'Mercedes-Benz', 'Lexus', 'Jaguar', 'Volvo', 'Land Rover', 'Porsche', 'Maserati', 'Alfa Romeo', 
                     'Aston Martin', 'Aston', 'Bentley', 'Ferrari', 'Lamborghini', 'Lotus', 'McLaren', 'Rolls-Royce', 'Holden','Tesla','Genesis',
                     'Saab', 'Rover','Hummer','Infiniti','Lotus','Polestar','Packard']
    if brand in luxury_brands:
           return 1
    else:
        return 0
cars['Luxury'] = cars['Brand'].apply(categorize_brand)
cars.head()

Note: we will ignore the model in our analysis. If we have enough data for a sepecific brand, it will be worth taking the model in consideration, when doing the analysis for an individual brand.

### Transmission

In [ ]:
cars['Transmission'].value_counts()

Let's analyse the entries where transmission = '-'

In [ ]:
cars[cars['Transmission'] == '-']

Those cars also have missing values in other columns. To simplify the process, we will drop those rows.

In [ ]:
cars = cars[cars['Transmission'] != '-']
cars['Transmission'].value_counts()

### Engine

In [ ]:
cars['Engine'].value_counts()

The column seems to follow a defined pattern, with some exceptions. Let's find the rows that do not match the pattern.

In [ ]:
not_matched_cons_cars = cars[cars['Engine'].str.match('\d+ cyl, \d[.]*\d* L')==False]
not_matched_cons_cars['Engine'].value_counts()

In [ ]:
# for simplicity, we will drop the rows with Engine = '4 cyl'
cars = cars[cars['Engine'] != '4 cyl']
cars['Engine'].value_counts()

In [ ]:
not_matched_cons_cars = cars[cars['Engine'].str.match('\d+ cyl, \d[.]*\d* L')==False]
not_matched_cons_cars['Engine'].value_counts()

Let's first analyze the '0 L' value

In [ ]:
cars.query('Engine == "0 L"')

It seems to correspond to electric cars. Let's confirm that, by filtering on cars with electric fueltype 

In [ ]:
cars.query('Engine == "0 L" and FuelType != "Electric"')

The only row obtained is a Nissan Leaf which is indeed electric. For convenience, we will make the correction there, by filling fueltype with the appropriate value:

In [ ]:
cars.loc[13378, 'FuelType'] = 'Electric'
cars.query('Engine == "0 L" and FuelType != "Electric"')

Let's analyze now the '-' value

In [ ]:
cars.query('Engine == "-"')

For those cars, Engine, Doors, Seats, FuelConsumption are missing. We will contact the DE team about that :). For now, we will drop the rows.

In [ ]:
cars = cars[cars['Engine'] != '-']
cars['Engine'].value_counts()

For the remaining columns we can extract the motor capacity

In [ ]:
cars['EngineMotorL'] = cars['Engine'].str.extract('(\d+(\.\d+)?)\s*L')[0]
# validate the extraction
cars['EngineMotorL'].isna().sum()

We will get rid of Engine column and use EngineMotorL instead.

### DriveType

In [ ]:
cars['DriveType'].value_counts()

5 rows are marked as "Other". We will drop those rows

In [ ]:

cars = cars[cars['DriveType'] != 'Other']
cars['DriveType'].value_counts()

### Fueltype

In [ ]:
cars['FuelType'].value_counts()

Fueltype value '-' is only found with non electric vehicles. 
Fueltype value 'Other' is also found with a non electric vehicle.

In [ ]:
cars.query("FuelType == '-' and EngineMotorL == '0'")


In [ ]:
cars.query("FuelType == 'Other'")

Assumption:
We will derive a new column from FuelType to categorize the fuel type. Hybrid and Electric will have the value 1, and all other values are mapped to 0.

In [ ]:
def categorize_fuel(fuel_type):
    
    electric_cat = ['Hybrid', 'Electric']
    if fuel_type in electric_cat:
        return 1
    else:
        return 0
cars['MotorType'] = cars['FuelType'].apply(categorize_fuel)
cars.head()

FuelType out, MotorType in.

In [ ]:
cars['MotorType'].value_counts()

### CylindersinEngine

In [ ]:
cars['CylindersinEngine'].value_counts()

A small transformation is applied, to extract the numbers at the beginning

In [ ]:
cars['CylindersinEngine'] = cars['CylindersinEngine'].str.extract('(\d+)')[0]
cars['CylindersinEngine'].value_counts()

### BodyType

In [ ]:
cars['BodyType'].value_counts()

While some make sense, we also have a Commercial category, people mover category, and other. We will drop this category. Depending on the model result, we might reconsider this decision.

### Fuel consumption
Extract and keep the fuel consumption

In [ ]:
cars["FuelConsumption"].value_counts()

In [ ]:
not_matched_cons_cars = cars[cars['FuelConsumption'].str.match('\d+[.]*\d* L / 100 km')==False]
not_matched_cons_cars

Extract the fuel consumption data in a new column.

In [ ]:
cars['FuelConsumptionValue'] = cars['FuelConsumption'].str.extract('(\d+(\.\d+)?)')[0]
cars['FuelConsumptionValue'].value_counts()

In [ ]:
cars["FuelConsumptionValue"].isna().sum()

In [ ]:
# drop rows with FuelConsumptionValue = NaN
cars = cars.dropna(subset=['FuelConsumptionValue'])

In [ ]:
# validate there is no missing data in the new column
cars["FuelConsumptionValue"].isna().sum()

### Kilometres

In [ ]:
cars['Kilometres'].value_counts()

2 values are not correct. Check the cars associated with those values.

In [ ]:
# 1
cars[cars['Kilometres'] == '- / -']

A small analysis shows that those cars correspond to new cars, year 2023. 

In [ ]:
# validate that the rows with Kilometres = '- / -' have year = 2023
cars.query('Kilometres == "- / -"')['Year'].value_counts()

We will then replace the Kilometres = '- / -' with 0

In [ ]:
cars.loc[cars['Kilometres'] == '- / -', 'Kilometres'] = 0

In [ ]:
# 2
cars[cars['Kilometres'] == '-']

In [ ]:
# validate that the rows with Kilometres = '-' have usedornew = 'New'
cars.query('Kilometres == "-"')['Year'].value_counts()

We do not have a default value to include in this case. We will remove those entries from the dataset.

In [ ]:
# drop those rows where kilometre = '-'
cars = cars[cars['Kilometres'] != '-']

We will not consider the color and the location for the analysis

### Doors

In [ ]:
cars['Doors'].value_counts()

In [ ]:
cars['Doors'].isna().sum()

Extract the doors value at the beginning

In [ ]:
cars['Doors'] = cars['Doors'].str.extract('(\d+)')[0]
cars['Doors'].value_counts()

### Seats

In [ ]:
cars['Seats'].value_counts()

In [ ]:
cars['Seats'].isna().sum()

In [ ]:
# drop rows with Seats = NaN
cars = cars.dropna(subset=['Seats'])
cars['Seats'].value_counts()

Extract the seats value at the beginning

In [ ]:
# extract the number of seats from the Seats column
cars['Seats'] = cars['Seats'].str.extract('(\d+)')[0]
cars['Seats'].value_counts()

In [ ]:
cars['Price'].value_counts()

We notice a value 'POA'for the price

In [ ]:
cars[cars['Price']=='POA']

We drop the rows with price = 'POA'

In [ ]:
cars = cars[cars['Price'] != 'POA']

In [ ]:
cars.info()

## DATA PREPARATION


In [ ]:
cars.columns

In [ ]:
columns_to_drop = ['Brand', 'Model', 'Title', 'Car/Suv','Engine','FuelType','FuelConsumption','ColourExtInt','Location','BodyType']
cars_analysis = cars.drop(columns=columns_to_drop)
cars_analysis.info()

In [ ]:
cars_analysis[['Price','Kilometres','CylindersinEngine','Doors','Seats','FuelConsumptionValue','EngineMotorL']] = cars_analysis[['Price','Kilometres','CylindersinEngine','Doors','Seats','FuelConsumptionValue','EngineMotorL']].astype(float)

In [ ]:
cars_analysis.info()